# AIOps Automation Demo (End-to-End)

Creates incident/runbook memories, runs `memory.search`, and demonstrates SelfModel reliability sampling (admin-only).

If you haven’t run the stack + seed yet, run the Quickstart notebook first.

In [1]:
from __future__ import annotations

import os
import sys
import time
from pathlib import Path

import httpx


def find_repo_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for candidate in [p, *p.parents]:
        if (candidate / "docker-compose.yml").exists():
            return candidate
    raise RuntimeError("Could not find repo root (missing docker-compose.yml)")


ROOT = find_repo_root()
API_BASE = os.environ.get("NINAI_API_BASE", "http://localhost:8000")
API_V1 = os.environ.get("NINAI_API_V1", f"{API_BASE}/api/v1")

sdk_path = ROOT / "sdk" / "python"
if str(sdk_path) not in sys.path:
    sys.path.insert(0, str(sdk_path))

from ninai import NinaiClient

In [2]:
# Login as admin (required for SelfModel sampling endpoints)

ADMIN_EMAIL = os.environ.get("NINAI_ADMIN_EMAIL", "admin@ninai.dev")
ADMIN_PASSWORD = os.environ.get("NINAI_ADMIN_PASSWORD", "admin1234")

resp = httpx.post(
    f"{API_V1}/auth/login",
    json={"email": ADMIN_EMAIL, "password": ADMIN_PASSWORD},
    timeout=10.0,
)

if resp.status_code != 200:
    raise RuntimeError(
        f"Login failed ({resp.status_code}). Run ninai_quickstart.ipynb first. Body: {resp.text[:200]}"
    )

payload = resp.json()
access_token = payload["access_token"]
org_id = (payload.get("user") or {}).get("organization_id")

client = NinaiClient(access_token=access_token, base_url=API_V1, organization_id=org_id)
print("Logged in as admin; org_id:", org_id)

Logged in as admin; org_id: 4b67a388-5a87-4281-bb01-b076a6b335e2


In [4]:
# Create a couple AIOps memories

runbook = client.memories.create(
    content="Runbook: if 5xx rate increases, check deployment diff, then DB pool saturation, then Redis latency.",
    tags=["aiops", "runbook", "5xx"],
    scope="organization",
    scope_id=org_id,
)

incident = client.memories.create(
    content="Incident INC-2026-01: Elevated p95 latency after enabling feature flag X. Mitigation: roll back flag; root cause: N+1 query.",
    tags=["aiops", "incident", "latency"],
    scope="organization",
    scope_id=org_id,
)

print("Created:", runbook.id, incident.id)

Created: 94185d35-e79a-43f5-afb9-27db7ff2f134 4d83da0c-00a3-4412-812f-324022bc66e9


In [5]:
# Use memory.search tool to fetch evidence

res = client.tools.invoke(
    tool_name="memory.search",
    tool_input={"query": "latency", "limit": 5, "scope": "organization"},
    scope="organization",
    scope_id=org_id,
    justification="aiops-demo",
)

print("status:", res.status)
print("warnings:", res.warnings)
for row in (res.output or {}).get("results", []):
    print("-", row.get("tags"), row.get("summary"))

status: success
warnings: None
- ['aiops', 'runbook', 'latency'] Runbook: if API latency spikes, check Redis saturation and DB connection pool.
- ['aiops', 'runbook', '5xx'] Runbook: if 5xx rate increases, check deployment diff, then DB pool saturation, then Redis latency.
- ['aiops', 'incident', 'latency'] Incident INC-2026-01: Elevated p95 latency after enabling feature flag X. Mitigation: roll back flag; root cause: N+1 query.
